In [1]:
from IPython.display import HTML, display

display(HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
toggle the code of this notebook on/off &nbsp;<a href="javascript:code_toggle()">^</a>'''))

### PANORAMAS OF CINEMA online

# // SUBTITLES SEARCH
v0.02

In [2]:
# GLOBAL
app_http = 'http://ec2-3-127-150-33.eu-central-1.compute.amazonaws.com'
button_color = 'black' # #7d1b27

## SEARCH THE VIDEOTHEQUE

### > type your query

In [3]:
# DISPLAY WIDGETS 
from IPython.display import display, clear_output
from ipywidgets import interactive, interact, interactive_output
import ipywidgets as widgets

options = []
s=0.01
for i in range(100):
    options.append("%.2f"%s)
    s+=.01

text_q = widgets.Text(value='drops of blood', placeholder='', description='text')
confidence = widgets.SelectionRangeSlider(options=options, index=(0,99), description='confidence')
cat = widgets.Dropdown(options=[('videotheque', 0),
                                ('solaris mon amour', 1),
                                ('equilibrium of an oldboy', 2),
                                ('the 400 children of la haine', 3),
                                ('y tu mama tambien departed', 4)],
                                value=0, description='category')
max_results_q = widgets.Text(value='10', placeholder='', description='max results')
hlink = widgets.HTML(description='', value="&nbsp;&nbsp;&nbsp;&nbsp; <a target='_blank' href='https://panoramas-of-cinema.s3.eu-central-1.amazonaws.com/indexes/categories.txt'>></a>", placeholder='')
uiH = widgets.HBox([cat, hlink])
uiV = widgets.VBox([text_q, confidence, uiH, max_results_q])

display(uiV)


In [4]:
# REFRESH QUERY
import requests, ast

def format_text(query):
    this_query = query.replace(' ', '%20')
    return this_query
    
def refresh_query(event):
    clear_output()
    display(search_button)
    
    text = format_text(text_q.value)
    confX = confidence.value[0]
    confY = confidence.value[1]
    max_results = max_results_q.value
    this_cat = cat.value

    # 
    this_query = app_http+'/subs?'
    if text: this_query = this_query+'text=%s&confidence_min=%s&confidence_max=%s&category=%s'%(text,confX,confY,this_cat)
    if max_results: this_query = this_query+'&maxResults=%s'%max_results
    #print(this_query)
    
    #global my_text
    my_text = ast.literal_eval(requests.get(this_query).text)['body']
    if len(my_text) == 0:
        print ('nothing here')
        return 
    
    # BUTTON EVENT ------------------------------
    def on_button_clicked(event):
        global movie_2p
        global frame_2p
        this_button = id(event)
        for i, b in enumerate(buttons_text):
            this_id = id(b)
            if this_button == this_id:
                movie_2p = buttons_text[i].description
                frame_2p = '/frame-%06d.jpg'%float(buttons_text[i].tooltip)
        
    # DISPLAY THE RESULTS ------------------------------
    from ipywidgets import VBox, Label, Button
    
    # formatting
    texts_results = []
    for movie, timestamp, prev, this, nex in my_text:
        this_text = prev+' '+this+' '+nex
        texts_results+=[this_text]
        
    # widgets
    global buttons_text
    buttons_text = []
    
    items = []
    for i, t in enumerate(texts_results):
        button = Button(description=my_text[i][0], tooltip=str(my_text[i][1]))
        buttons_text.append(button)
        button.on_click( on_button_clicked )
        
        items.append(Label(' '))
        items.append(Label(t))
        hlink = widgets.HTML(description='', value="&nbsp;&nbsp;&nbsp;&nbsp; <a target='_blank' href='https://panoramas-of-cinema.s3.eu-central-1.amazonaws.com/subtitles_db%s/subs.txt'>></a>"%my_text[i][0], placeholder='')
        uiH = widgets.HBox([button, hlink])
        items.append(uiH)
            
    display( VBox(items) )

                    
search_button = widgets.Button(description="search", button_style='info')
search_button.style.button_color = button_color
display(search_button)
search_button.on_click(refresh_query)

Button(button_style='info', description='search', style=ButtonStyle(button_color='black'))

In [5]:
print('\n', '\n', '\n', '\n', '\n')

## PLAY A CLIP

### > click on a dialogue

In [6]:
# PLAY THE CLIP 
def play_clip(event):
    clear_output()
    display(ui1)

    global r
    clip_query = 'http://pocgetclip-env.eba-4phq6b4m.eu-central-1.elasticbeanstalk.com/clip?movie=%s&frame=%s'%(movie_2p[1:],frame_2p[7:-4])
    r = requests.get(clip_query).json()

    from IPython.display import Video
    display(Video(r['play'], height=300, html_attributes="controls autoplay"))
    print('%s%s'%(movie_2p[1:],frame_2p[6:-4]))
    
def play_before(event):
    clear_output()
    display(ui1)
    
    from IPython.display import Video
    display(Video(r['before'], height=300, html_attributes="controls autoplay"))
    print(movie_2p[1:])
    
def play_after(event):
    clear_output()
    display(ui1)
    
    from IPython.display import Video
    display(Video(r['after'], height=300, html_attributes="controls autoplay")) 
    print(movie_2p[1:])

# buttons box
play_button = widgets.Button(description="play", button_style='info')
play_button.style.button_color = button_color
playB_button = widgets.Button(description="<<<", button_style='info')
playB_button.style.button_color = button_color
playA_button = widgets.Button(description=">>>", button_style='info')
playA_button.style.button_color = button_color
ui1 = widgets.HBox([playB_button, play_button, playA_button])
display(ui1)


play_button.on_click(play_clip)
playB_button.on_click(play_before)
playA_button.on_click(play_after)

In [7]:
print('\n', '\n', '\n', '\n', '\n')

ETH Zurich &nbsp;&nbsp;&nbsp;&nbsp; Department of Architecture &nbsp;&nbsp;&nbsp;&nbsp; Chair for Digital Architectonics &nbsp;&nbsp;&nbsp;&nbsp; 2021